In [2]:
import sqlite3

# Определяем класс Professor
class Professor:
    def __init__(self, name, subject, attributes):
        self.name = name
        self.subject = subject
        self.attributes = attributes

# Определяем класс Node для дерева решений
class Node:
    def __init__(self, question=None):
        self.question = question
        self.yes_branch = None
        self.no_branch = None
        self.professor = None  # Ссылка на профессора, если это конечный узел

# Подключаемся к базе данных (или создаем ее, если она не существует)
conn = sqlite3.connect('professors.db')
cursor = conn.cursor()

# Создаем таблицы для преподавателей и вопросов
cursor.execute('''CREATE TABLE IF NOT EXISTS questions (
    id INTEGER PRIMARY KEY,
    question TEXT NOT NULL
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS professors (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    subject TEXT NOT NULL,
    attributes BLOB NOT NULL
)''')

# Заполняем таблицу вопросов, если она пуста
cursor.execute('SELECT COUNT(*) FROM questions')
if cursor.fetchone()[0] == 0:
    questions_data = [
        "Мужчина?",
        "Преподает математику?",
        "Преподает физику?",
        "Преподает биологию?",
        "Преподает информатику?",
        "Преподает химию?",
        "Имеет степень магистра?",
        "Имеет степень доктора?",
        "Преподает в университете?"
    ]
    cursor.executemany('INSERT INTO questions (question) VALUES (?)', [(q,) for q in questions_data])

# Заполняем таблицу преподавателей, если она пуста
cursor.execute('SELECT COUNT(*) FROM professors')
if cursor.fetchone()[0] == 0:
    professors_data = [
        ("Преподаватель по математике 1", "математика", [1, 1, 0, 0, 0, 0, 1, 0, 0]),
        ("Преподаватель по математике 2", "математика", [1, 1, 0, 0, 0, 0, 0, 1, 0]),
        ("Преподаватель по математике 3", "математика", [1, 1, 0, 0, 0, 0, 1, 1, 1]),
        ("Преподаватель по физике 1", "физика", [1, 0, 1, 0, 0, 0, 0, 1, 0]),
        ("Преподаватель по физике 2", "физика", [1, 0, 1, 0, 0, 0, 0, 0, 1]),
        ("Преподаватель по биологии 1", "биология", [0, 0, 0, 1, 0, 0, 0, 1, 0]),
        ("Преподаватель по биологии 2", "биология", [0, 0, 0, 1, 0, 0, 0, 0, 1]),
        ("Преподаватель по химии 1", "химия", [1, 0, 0, 0, 0, 1, 0, 1, 0]),
        ("Преподаватель по химии 2", "химия", [0, 0, 0, 0, 0, 1, 0, 0, 1]),
        ("Преподаватель по информатике 1", "информатика", [1, 0, 0, 0, 1, 0, 0, 1, 0]),
        ("Преподаватель по информатике 2", "информатика", [1, 0, 0, 0, 1, 0, 0, 0, 1]),
    ]

    for name, subject, attributes in professors_data:
        cursor.execute('INSERT INTO professors (name, subject, attributes) VALUES (?, ?, ?)', 
                       (name, subject, str(attributes)))

def get_questions():
    cursor.execute('SELECT question FROM questions')
    return [row[0] for row in cursor.fetchall()]

def get_professors():
    cursor.execute('SELECT name, subject, attributes FROM professors')
    return [Professor(name=row[0], subject=row[1], attributes=eval(row[2])) for row in cursor.fetchall()]

def build_decision_tree(professors, questions):
    root = Node()  # Корень дерева
    if len(professors) == 1:
        # Если найден только один профессор, назначаем его
        root.professor = professors[0]
        return root
    
    if not questions:
        return root  # Нет вопросов, возвращаем пустой узел

    # Ищем подходящий вопрос
    for question_index in range(len(questions)):
        # Сначала фильтруем профессоров по текущему вопросу
        yes_professors = [prof for prof in professors if prof.attributes[question_index] == 1]
        no_professors = [prof for prof in professors if prof.attributes[question_index] == 0]

        # Если оба списка не пусты, мы можем задать этот вопрос
        if yes_professors and no_professors:
            root.question = questions[question_index]

            # Рекурсивно строим ветви
            root.yes_branch = build_decision_tree(yes_professors, questions)
            root.no_branch = build_decision_tree(no_professors, questions)

            return root

    return root  # Если нет подходящих вопросов, возвращаем пустой узел

def ask_question(node):
    if node.professor:
        print(f"Вы выбрали: {node.professor.name} (предмет: {node.professor.subject})")
        return

    answer = input(f"{node.question} (да/нет): ").strip().lower()

    if answer == 'да':
        ask_question(node.yes_branch)
    elif answer == 'нет':
        ask_question(node.no_branch)
    else:
        print("Пожалуйста, отвечайте только 'да' или 'нет'.")
        ask_question(node)

def add_professor():
    name = input("Введите имя нового преподавателя: ")
    subject = input("Введите предмет, который будет преподавать: ").strip()
    attributes = []
    
    for question in get_questions():
        answer = input(f"{question} (да/нет): ").strip().lower()
        if answer == 'да':
            attributes.append(1)
        elif answer == 'нет':
            attributes.append(0)
        else:
            print("Пожалуйста, отвечайте только 'да' или 'нет'.")
            return add_professor()  # Повторяем добавление

    cursor.execute('INSERT INTO professors (name, subject, attributes) VALUES (?, ?, ?)', 
                   (name, subject, str(attributes)))
    print(f"{name} успешно добавлен в базу данных!")

# Запускаем Акинатора
print("Добро пожаловать в Акинатора! Давайте попробуем угадать преподавателя.")
questions = get_questions()
professors = get_professors()
decision_tree = build_decision_tree(professors, questions)
ask_question(decision_tree)  # Начинаем с корня дерева

# Закрываем соединение с базой данных
conn.commit()  # Сохраняем изменения
conn.close()


Добро пожаловать в Акинатора! Давайте попробуем угадать преподавателя.


AttributeError: 'NoneType' object has no attribute 'professor'